Connecting to Augur

In [8]:
import os
print(os.environ.get('PG_USER'))

None


In [5]:
import pandas as pd
import json
import sqlalchemy as salc
import psycopg2
import os

database_connection_string = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(
                              os.environ.get('PG_USER'),
                              os.environ.get('PG_PASS'),
                              os.environ.get('PG_HOST'),
                              os.environ.get('PG_PORT'),
                              os.environ.get('PG_DB')
                            )

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

ModuleNotFoundError: No module named 'psycopg2'

## Activity Metrics
- SQL query for github activity metrics
- Focused on the increment in activity (star/fork/watch/committer/commit/issue) over time to prevent from old repo having lots of accumulated activity.
    -  Assigned different weights for #increase_in_stars, #increase_in_forks, #increase_in_watch, #increase_in_committer, #increase_in_commit, #increase_in_issue, #increase_in_pr, #increase_in_pr_open, #increase_in_pr_close, #increase_in_pr_merge, then sum them together

In [ ]:
dframe = pd.DataFrame()

repo_query = salc.sql.text(f"""
SELECT x.repo_id,
       x.rg_name,
       x.repo_name,
       last_updated,
       DATE(last_updated),
       to_char(last_updated, 'DAY'),
       EXTRACT(year FROM last_updated) AS "Year",
       EXTRACT(month FROM last_updated) AS "month",
       extract(hour from last_updated) AS "hour",
       x.increase_committer,
       x.increase_pr_open,
       x.increase_commit,
       (x.increase_committer + x.increase_pr_open + x.increase_pr_close + x.increase_pr_merge + x.increase_issue + x.increase_pr + x.increase_star + x.increase_fork)*10 AS total
            FROM(
        SELECT 
            rg.repo_group_id,
            rg.rg_name,
            r.repo_id,
            r.repo_name,
            /*ri.license,*/
            CASE
                WHEN r.repo_id - lag(r.repo_id) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated) = 0 THEN 
                (ri.stars_count - lag(ri.stars_count) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated)) * 0.01
            END
                AS increase_star,
            CASE
                WHEN r.repo_id - lag(r.repo_id) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated) = 0 THEN 
                (ri.fork_count - lag(ri.fork_count) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated)) * 0.2
            END
                AS increase_fork,
            CASE
                WHEN r.repo_id - lag(r.repo_id) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated) = 0 THEN 
                (ri.watchers_count - lag(ri.watchers_count) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated)) * 0.1
            END
                AS increase_watch,
            CASE
                WHEN r.repo_id - lag(r.repo_id) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated) = 0 THEN 
                (ri.committers_count - lag(ri.committers_count) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated)) *1.6
            END
                AS increase_committer,
            CASE
                WHEN r.repo_id - lag(r.repo_id) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated) = 0 THEN 
                (ri.commit_count - lag(ri.commit_count) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated)) * 1.3
            END
                AS increase_commit,
            CASE
                WHEN r.repo_id - lag(r.repo_id) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated) = 0 THEN 
                (ri.issues_count - lag(ri.issues_count) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated)) * 0.5
            END
                AS increase_issue,
            CASE
                WHEN r.repo_id - lag(r.repo_id) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated) = 0 THEN 
                (ri.pull_request_count - lag(ri.pull_request_count) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated)) * 1
            END
                AS increase_pr,
            CASE
                WHEN r.repo_id - lag(r.repo_id) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated) = 0 THEN 
                (ri.pull_requests_open - lag(ri.pull_requests_open) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated)) * 1.2
            END
                AS increase_pr_open,
            CASE
                WHEN r.repo_id - lag(r.repo_id) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated) = 0 THEN 
                (ri.pull_requests_closed - lag(ri.pull_requests_closed) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated)) * 1.5
            END
                AS increase_pr_close,
            CASE
                WHEN r.repo_id - lag(r.repo_id) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated) = 0 THEN 
                (ri.pull_requests_merged - lag(ri.pull_requests_merged) over (order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated)) * 1.8    
            END
                AS increase_pr_merge,
            ri.last_updated,
            CASE
                WHEN EXTRACT(YEAR FROM ri.last_updated) < 2022 THEN 'far away'
                WHEN EXTRACT(YEAR FROM ri.last_updated) >= 2022 THEN 'recent'
            END
                AS segment,
            EXTRACT(year FROM last_updated) AS "Year",
            EXTRACT(month FROM last_updated) AS "month" 
        FROM REPO r
            LEFT JOIN repo_groups rg
            ON rg.repo_group_id = r.repo_group_id
            LEFT join repo_info ri 
            on r.repo_id = ri.repo_id 
        /*where rg.rg_name = 'agroal'*/
        order by rg.repo_group_id ASC, r.repo_id ASC, ri.last_updated) AS x
""")

dframe = pd.read_sql(repo_query, con=engine)
dframe.head()

In [ ]:
# Fill all NA value into zero
dframe = dframe.fillna(0)
# calculating activeness percentage based on org and repo_name
df2 = dframe.groupby(['rg_name', 'repo_name']).agg({'total': 'sum'})
df3 = df2.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))
df4 = df3['total'].to_frame().reset_index()
df4 = df4[df4['total'] != 0.0]
df4.head()

# extract column 'month' and 'percentage', copy it as another dataframe
# dft = dtest[['month', 'percentage']].copy()
# dft = dframe['percentage'].copy()

In [ ]:
t = df4[df4['rg_name'] == 'openshift']
t[t['repo_name'] == 'docker-distribution']

### Bar chart

In [ ]:
import plotly.express as px

fig = px.bar(df4, x="rg_name", y="total", color="repo_name", text="repo_name")
fig.show()

### Pie chart

In [ ]:
drepo = df4[df4['rg_name'] == 'kubernetes']

import plotly.express as px
fig_pie = px.pie(data_frame=drepo, names='repo_name', values='total')
fig_pie.show()

### Plot for org Density by Activities

In [ ]:
dframe = dframe.fillna(0)
# calculating activeness percentage based on org and repo_name
df2 = dframe.groupby(['rg_name', 'repo_name']).agg({'total': 'sum'})
ho = df2['total'].to_frame().reset_index()


hoo = ho.groupby(['rg_name']).agg({'total': 'sum'})
drank = hoo.reset_index()
drank.sort_values(by = 'total', ascending=False).reset_index()
drank.head()

In [ ]:
import plotly.express as px

fig = px.bar(drank, x="total", y="rg_name", orientation='h')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

In [ ]:
drank[drank['rg_name'] == 'GNOME']

### Comments

In [ ]:
# Activity
dframe_msg = pd.DataFrame()

message_query = salc.sql.text(f"""
    SELECT x.repo_id,
            x.repo_name,
            x.timeyear,
            x.timemonth,
            x.timedate,
            COUNT(*) AS num_of_comment
    FROM(
        SELECT m.repo_id, 
                r.repo_name,
                EXTRACT (year FROM msg_timestamp) AS timeyear,
                EXTRACT (month FROM msg_timestamp) AS timemonth,
                EXTRACT (day FROM msg_timestamp) AS timedate
                FROM message m 
            left join repo r 
            on m.repo_id = r.repo_id 
            where m.repo_id is not null
            limit 1000
        ) AS x
    GROUP BY x.repo_id, x.repo_name, x.timeyear, x.timemonth, x.timedate
    ORDER BY x.repo_id, x.timeyear, x.timemonth, x.timedate
""")

dframe_msg = pd.read_sql(message_query, con=engine)
dframe_msg.head()

In [ ]:
dframe_msg = dframe_msg.fillna(0)
print(len(dframe_msg))

In [ ]:
import plotly.express as px

fig = px.bar(dframe_msg, x = 'timeyear', y = 'num_of_comment', color='repo_name',
                labels={'num_of_comment': 'Number Of Comments'},
                height = 400)
fig.show()

- Calculate number of comments for a repo by identifying unique contributor and within what timeframe

In [ ]:
dframe_unq_msg_cntrb = pd.DataFrame()

unq_msg_cntrb_query = salc.sql.text(f"""
	SELECT x.repo_id,
			x.repo_name,
			x.datetime,
			COUNT(x.msg_text) AS num_of_comments,
			COUNT(distinct x.cntrb_id) AS num_of_unique_msg_cntrb
		FROM(
			SELECT m.repo_id,
					r.repo_name,
					m.cntrb_id,
					m.msg_text,
					TO_CHAR(msg_timestamp  :: DATE, 'yyyy-mm-dd') AS datetime
			FROM message m
			LEFT JOIN repo r
				ON m.repo_id = r.repo_id
			WHERE m.repo_id is not null
			ORDER BY m.repo_id, datetime
			) AS x
		GROUP BY x.datetime, x.repo_id, x.repo_name
	ORDER BY x.repo_id, x.datetime
""")

dframe_unq_msg_cntrb = pd.read_sql(unq_msg_cntrb_query, con=engine)
dframe_unq_msg_cntrb.head()

In [ ]:
dtest = dframe_unq_msg_cntrb[dframe_unq_msg_cntrb['repo_id'] == 24441]
dtest.head()

Number of Comments V.S. Number of Unique Comment Contributors by day

In [ ]:
import plotly.graph_objects as go

benchmark_value = dtest['num_of_comments'].mean()


fig = go.Figure()
fig.add_trace(go.Bar(
    x=dtest['datetime'],
    y=dtest['num_of_comments'],
    name='Number Of Comments',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=dtest['datetime'],
    y=dtest['num_of_unique_msg_cntrb'],
    name='Number Of Unique Contributors',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.add_hline(y=benchmark_value, annotation_text=f"{round(benchmark_value, 2)}: Avg for # of comments", line_dash='dot', annotation_font_size=20)
fig.add_hline(y=dtest['num_of_unique_msg_cntrb'].mean(), annotation_text=f"{round(dtest['num_of_unique_msg_cntrb'].mean(),2)}: Avg for # of unique comment contributors")
fig.update_layout(barmode='group', xaxis_tickangle=-45, title='Number of Comments V.S. Number of Unique Comment Contributors by day - operate-first-twitter repo')
fig.show()

In [ ]:
dframe_unq_msg_cntrb = pd.DataFrame()

unq_msg_cntrb_query = salc.sql.text(f"""
	SELECT x.repo_id,
			x.repo_name,
			x.yearmonth,
			COUNT(x.msg_text) AS num_of_comments,
			COUNT(distinct x.cntrb_id) AS num_of_unique_msg_cntrb
		FROM(
			SELECT m.repo_id,
					r.repo_name,
					m.cntrb_id,
					m.msg_text,
					CAST(EXTRACT(YEAR FROM msg_timestamp) AS text) || '-' || CAST(EXTRACT(MONTH FROM msg_timestamp) AS text) AS yearmonth
			FROM message m
			LEFT JOIN repo r
				ON m.repo_id = r.repo_id
			WHERE m.repo_id is not null
			ORDER BY m.repo_id, yearmonth
			) AS x
		GROUP BY x.repo_id, x.repo_name, x.yearmonth
	ORDER BY x.repo_id, x.yearmonth
""")

dframe_unq_msg_cntrb = pd.read_sql(unq_msg_cntrb_query, con=engine)
dframe_unq_msg_cntrb.head()

In [ ]:
dhey = dframe_unq_msg_cntrb[dframe_unq_msg_cntrb['repo_name'] == 'kubernetes']
dhey.head()

Number of Comments V.S. Number of Unique Comment Contributors by month -- Kubernetes repo

In [ ]:
benchmark_value = dhey['num_of_comments'].mean()
dhey['benchmark_score'] = dhey['num_of_comments'] - benchmark_value
benchmark_value
dhey.head()

In [ ]:
import plotly.graph_objects as go

benchmark_value = dhey['num_of_comments'].mean()


fig = go.Figure()
fig.add_trace(go.Bar(
    x=dhey['yearmonth'],
    y=dhey['num_of_comments'],
    name='Number Of Comments',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=dhey['yearmonth'],
    y=dhey['num_of_unique_msg_cntrb'],
    name='Number Of Unique Contributors',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.add_hline(y=benchmark_value, annotation_text=f" {round(benchmark_value,2)}: Avg for # of comments", line_dash='dot', annotation_font_size=20)
fig.add_hline(y=dhey['num_of_unique_msg_cntrb'].mean(), annotation_text= f" {round(dhey['num_of_unique_msg_cntrb'].mean(),2)} : Avg for # of unique comment contributors")
fig.update_layout(barmode='group', xaxis_tickangle=-45, title='Number of Comments V.S. Number of Unique Comment Contributors by month - Kubernetes repo')
fig.show()

# Community
### Contributors
Defined as committers and authors

In [ ]:
df_pr_committers = pd.DataFrame()
df_pr_authors = pd.DataFrame()

committer_query = salc.sql.text(f"""
    SELECT x.rg_name,
            x.repo_id,
            x.repo_name,
            x.cmt_committer_date,
            COUNT(x.cmt_id) AS num_of_commit,
            COUNT(DISTINCT x.cmt_committer_raw_email) AS num_of_unique_commit
        FROM(
            SELECT rg.rg_name,
                    c.repo_id,
                    r.repo_name,
                    c.cmt_id,
                    c.cmt_committer_raw_email,
                    c.cmt_committer_date 
            FROM commits c 
            LEFT JOIN repo r
                ON c.repo_id = r.repo_id
            left join repo_groups rg
                on r.repo_group_id = rg.repo_group_id
            WHERE c.repo_id is not null
                AND c.cmt_committer_date >= '2020-01-01'
            ORDER BY r.repo_group_id, c.repo_id, c.cmt_committer_date
            ) as x
        GROUP BY x.cmt_committer_date, x.rg_name, x.repo_id, x.repo_name
    ORDER BY x.rg_name, x.repo_id, x.cmt_committer_date
""")


author_query = salc.sql.text(f"""
    SELECT x.rg_name,
            x.repo_id,
            x.repo_name,
            x.cmt_author_date,
            COUNT(x.cmt_id) AS num_of_author,
            COUNT(DISTINCT x.cmt_author_raw_email) AS num_of_unique_author
        FROM(
            SELECT rg.rg_name,
                    c.repo_id,
                    r.repo_name,
                    c.cmt_id,
                    c.cmt_author_raw_email,
                    c.cmt_author_date 
            FROM commits c 
            LEFT JOIN repo r
                ON c.repo_id = r.repo_id
            left join repo_groups rg
                on r.repo_group_id = rg.repo_group_id
            WHERE c.repo_id is not null
                AND c.cmt_committer_date >= '2020-01-01'
            ORDER BY r.repo_group_id, c.repo_id, c.cmt_author_date
            ) as x
        GROUP BY x.cmt_author_date, x.rg_name, x.repo_id, x.repo_name
    ORDER BY x.rg_name, x.repo_id, x.cmt_author_date
""")

In [ ]:
df_pr_committers = pd.read_sql(committer_query, con=engine)
df_pr_committers.head()

In [ ]:
df_pr_authors = pd.read_sql(author_query, con=engine)
df_pr_authors.head()

Number of committers V.S. Number of unique committers over time -- Operate First Twitter repo

In [ ]:
dcommit = df_pr_committers[df_pr_committers['rg_name'] == 24441]
dcommit.head()

In [ ]:
import plotly.graph_objects as go

benchmark_value = df_pr_committers['num_of_commit'].mean()


fig = go.Figure()
fig.add_trace(go.Bar(
    x=df_pr_committers['cmt_committer_date'],
    y=df_pr_committers['num_of_commit'],
    name='Number Of Committers',
    base='repo_name',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=df_pr_committers['cmt_committer_date'],
    y=df_pr_committers['num_of_unique_commit'],
    name='Number Of Unique Committers',
    base='repo_name',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.add_hline(y=benchmark_value, annotation_text=f"{round(benchmark_value, 2)}: Avg for # of Committers", line_dash='dot', annotation_font_size=20)
fig.add_hline(y=df_pr_committers['num_of_unique_commit'].mean(), annotation_text=f"{round(df_pr_committers['num_of_unique_commit'].mean(),2)}: Avg for # of unique Committers")
fig.update_layout(barmode='group', xaxis_tickangle=-45, title='Number of Committers V.S. Number of Unique Committers Contributors by day - operate-first-twitter repo')
fig.show()

Number of Authors V.S. Number of unique Authors over time -- Operate First Twitter repo

In [ ]:
dauthor = df_pr_authors[df_pr_authors['repo_id'] == 24441]
dauthor.head()

In [ ]:
import plotly.graph_objects as go

benchmark_value = dauthor['num_of_author'].mean()


fig = go.Figure()
fig.add_trace(go.Bar(
    x=dauthor['cmt_author_date'],
    y=dauthor['num_of_author'],
    name='Number Of Authors',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=dauthor['cmt_author_date'],
    y=dauthor['num_of_unique_author'],
    name='Number Of Unique Authors',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.add_hline(y=benchmark_value, annotation_text=f"{round(benchmark_value, 2)}: Avg for # of Authors", line_dash='dot', annotation_font_size=20)
fig.add_hline(y=dauthor['num_of_unique_author'].mean(), annotation_text=f"{round(dauthor['num_of_unique_author'].mean(),2)}: Avg for # of unique Authors")
fig.update_layout(barmode='group', xaxis_tickangle=-45, title='Number of Authors V.S. Number of Unique Authors Contributors by day - operate-first-twitter repo')
fig.show()

# Performance
## PR
SQL query for github performance metrics
- whether the PR is closed or open -> status
- time required to close an PR -> duration
- how many days has passed since the ticket is closed -> exp decay

In [ ]:
dframe_pr = pd.DataFrame()

pr_query = salc.sql.text(f"""
/*
1. whether the PR is closed or open -> status
2. time required to close an PR -> duration
3. Only the data from 2021 (?)
4. how many days has passed since the ticket is closed -> exp decay
*/
SELECT x.repo_group_id,
		rg.rg_name,
        x.repo_id,
        x.repo_name,
		x.close_duration,
		x.exp_decay,
		count(pull_request_id) AS num,
		close_duration*count(pull_request_id) + exp_decay*count(pull_request_id) as total
		FROM(
			SELECT pull_request_id,
                    pull_requests.repo_id,
                    r.repo_name,
                    r.repo_group_id,
				    pr_src_state,
				    (pr_closed_at - pr_created_at) AS pull_request_duration,
				   CASE 
				   	WHEN pull_requests.pr_closed_at - pull_requests.pr_created_at <= INTERVAL '15 days' THEN 1
				   	WHEN pull_requests.pr_closed_at - pull_requests.pr_created_at <= INTERVAL '30 days' THEN 0.66
				   	WHEN pull_requests.pr_closed_at - pull_requests.pr_created_at <= INTERVAL '60 days' THEN 0.33
				   	WHEN pull_requests.pr_closed_at - pull_requests.pr_created_at > INTERVAL '90 days' THEN 0.1
				   	WHEN pull_requests.pr_closed_at IS NULL AND NOW() - pull_requests.pr_created_at < INTERVAL '45 days' THEN 0.5
				   	ELSE 0
				   END
				   AS close_duration,
				   NOW() - pull_requests.pr_closed_at AS "time_passed",
				   CASE 
				   	WHEN NOW() - pull_requests.pr_closed_at < INTERVAL '30 days' THEN 1
				   	WHEN NOW() - pull_requests.pr_closed_at < INTERVAL '60 days' THEN 0.9
				   	WHEN NOW() - pull_requests.pr_closed_at < INTERVAL '90 days' THEN 0.9*0.9
				   	WHEN NOW() - pull_requests.pr_closed_at < INTERVAL '120 days' THEN 0.9*0.9*0.9
				   	WHEN NOW() - pull_requests.pr_closed_at < INTERVAL '150 days' THEN 0.9*0.9*0.9*0.9
				   	WHEN NOW() - pull_requests.pr_closed_at < INTERVAL '180 days' THEN 0.9*0.9*0.9*0.9*0.9
				   	WHEN NOW() - pull_requests.pr_closed_at < INTERVAL '210 days' THEN 0.9*0.9*0.9*0.9*0.9*0.9
				   	WHEN NOW() - pull_requests.pr_closed_at < INTERVAL '240 days' THEN 0.9*0.9*0.9*0.9*0.9*0.9*0.9
				   	WHEN NOW() - pull_requests.pr_closed_at < INTERVAL '270 days' THEN 0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9
				   	WHEN NOW() - pull_requests.pr_closed_at < INTERVAL '300 days' THEN 0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9
				   	WHEN NOW() - pull_requests.pr_closed_at < INTERVAL '330 days' THEN 0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9
				   	WHEN NOW() - pull_requests.pr_closed_at < INTERVAL '360 days' THEN 0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9
				   	ELSE 0
				   END
				   AS exp_decay,
				   pull_requests.pr_closed_at
			FROM pull_requests
			/*WHERE EXTRACT(YEAR FROM pull_requests.pr_closed_at) >= 2022*/
            LEFT JOIN repo r
                ON r.repo_id = pull_requests.repo_id
			ORDER BY repo_id 
		) AS x
		left join repo_groups rg 
			on rg.repo_group_id = x.repo_group_id
	GROUP BY x.repo_id, x.repo_name, x.repo_group_id, rg.rg_name, close_duration, exp_decay
	order by x.repo_id

""")

dframe_pr = pd.read_sql(pr_query, con=engine)

dframe_pr.head()

In [ ]:
dframe_pr_g = dframe_pr.groupby('repo_id')['total'].sum()
dframe_pr_g.head()

## Issue
- SQL query for github performance metrics
- time required to close an issue -> duration
- how many days has passed since the ticket is closed -> exp decay

In [ ]:
issue_query = salc.sql.text(f"""
SELECT repo_id,
		close_duration,
		exp_decay,
		count(issue_id) AS num,
		close_duration*exp_decay*count(issue_id) AS total
		FROM(
			SELECT repo_id,
				   issue_id,
				   issue_state,
				   (closed_at - created_at) AS issue_close_duration,
				   /*(updated_at - created_at) as issue_update_duration,*/
				   CASE 
					   	WHEN i.closed_at - i.created_at <= interval '30 days' THEN 1
					   	WHEN i.closed_at - i.created_at <= interval '60 days' THEN 0.66
					   	WHEN i.closed_at - i.created_at <= interval '90 days' THEN 0.33
					   	WHEN i.closed_at - i.created_at > interval '90 days' THEN 0.1
						/* the issue that has recently been opened*/
					   	when i.closed_at IS NULL AND NOW() - i.created_at < interval '45 days' THEN 0.5
					   	ELSE 0
				   END
				   AS close_duration,
				   i.created_at,
				   i.closed_at,
				   NOW() - i.closed_at AS "time_passed_after_closing",
				   CASE 
				   	WHEN NOW() - i.closed_at < interval '30 days' THEN 1
				   	WHEN NOW() - i.closed_at < interval '60 days' THEN 0.9
				   	WHEN NOW() - i.closed_at < interval '90 days' THEN 0.9*0.9
				   	WHEN NOW() - i.closed_at < interval '120 days' THEN 0.9*0.9*0.9
				   	WHEN NOW() - i.closed_at < interval '150 days' THEN 0.9*0.9*0.9*0.9
				   	WHEN NOW() - i.closed_at < interval '180 days' THEN 0.9*0.9*0.9*0.9*0.9
				   	WHEN NOW() - i.closed_at < interval '210 days' THEN 0.9*0.9*0.9*0.9*0.9*0.9
				   	WHEN NOW() - i.closed_at < interval '240 days' THEN 0.9*0.9*0.9*0.9*0.9*0.9*0.9
				   	WHEN NOW() - i.closed_at < interval '270 days' THEN 0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9
				   	WHEN NOW() - i.closed_at < interval '300 days' THEN 0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9
				   	WHEN NOW() - i.closed_at < interval '330 days' THEN 0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9
				   	WHEN NOW() - i.closed_at < interval '360 days' THEN 0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9*0.9
				   	ELSE 0
				   END
				   AS exp_decay
			FROM issues i
			WHERE extract(year from i.created_at) >= 2022
			ORDER BY repo_id
			) AS x
	GROUP BY repo_id, close_duration, exp_decay
	order by repo_id
""")

dframe_issue = pd.read_sql(issue_query, con=engine)

dframe_issue.head()

In [ ]:
dframe_issue_g = dframe_issue.groupby('repo_id')['total'].sum()
dframe_issue_g.head()

## Performance Metrics
- Evaluating the duration from issue open to close as issue_close_duration column
- Issue_close_duration_day column: extract day from issue_close_duration

In [ ]:
issue2_query = salc.sql.text(f"""	
			SELECT repo_id,
				   issue_state,
				   (closed_at - created_at) AS issue_close_duration,
				   extract(day from closed_at - created_at) AS issue_close_duration_day,
				   comment_count
			FROM issues i
			WHERE extract(year from i.created_at) >= 2022
			ORDER BY repo_id
""")

dtest = pd.read_sql(issue2_query, con=engine)

In [ ]:
dtest.head()

Extract only issue_close_duration_day and comment_count column from the original dataframe and make a copy

In [ ]:
frame = dtest[['issue_close_duration_day', 'comment_count']].copy()
frame = frame.fillna(0)
frame.head()

## Elbow method...
Apply KMenas clustering on the duration_day and comment_count column for segmentation on issue being closed within how many days should be segmented into how many differenct groups, what are each decay rate...
- Number of clusters (n = 4): number of groups for different performance (active performance/ mild performance/ poor performance/ low performance)
- Cluster size (179995, 13545, 3011, 103): The threshold for the four groups?
- Decay rate (slope in the curve?): how much weight should be given to active performance? Mild performance? Poor performance? Low performance?

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,12)).fit(frame)
visualizer.show()

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, init='k-means++', random_state=0).fit(frame)

In [ ]:
print(kmeans.labels_)
print(kmeans.inertia_)
print(kmeans.n_iter_)
print(kmeans.cluster_centers_)

k-means clustering has four clusters of size 179995, 13545, 3011, and 103, which in our case menas that there's 179995 active performance, 13545 mild performace, and 3011 poor performance...

In [ ]:
from collections import Counter
Counter(kmeans.labels_)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.scatterplot(data=frame, x="issue_close_duration_day", y="comment_count", hue=kmeans.labels_)
plt.show()

Question: I used commnet count as y axis, but not sure if that make sense at all

## Activity plot (WIP)